<a href="https://colab.research.google.com/github/juniismai02/tesis-kanker-kulit-tf-vggnet-mobilenet-juniismail-/blob/main/boosting_algorithm_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy catboost scipy
!pip cache purge

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: catboost 1.2.7
Uninstalling catboost-1.2.7:
  Successfully uninstalled catboost-1.2.7
Found existing installation: scipy 1.15.2
Uninstalling scipy-1.15.2:
  Successfully uninstalled scipy-1.15.2
Files removed: 20


In [ ]:
!pip install numpy catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 13.8 MB/s eta 0:00:00


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from google.colab import files

# Load Dataset

In [ ]:
file_path = '/content/drive/MyDrive/tesis/data_penjualan_tiket_multi_rute.xlsx'
df_2023 = pd.read_excel(file_path, sheet_name='2023')
df_2024 = pd.read_excel(file_path, sheet_name='2024')

# Data Cleaning and Preparation

In [ ]:
# Convert the price to numeric
for col in df_2024.columns[1:]:
    df_2024[col] = pd.to_numeric(df_2024[col], errors='coerce')

# add year coloum
df_2023['Tahun'] = 2023
df_2024['Tahun'] = 2024

# Merge dataset
df_combined = pd.concat([df_2023, df_2024], ignore_index=True)

# Ordered by year and month
month_order = ['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni',
               'Juli', 'Agustus', 'September', 'Oktober', 'November', 'Desember']
df_combined['Bulan'] = pd.Categorical(df_combined['Bulan'], categories=month_order, ordered=True)
df_combined.sort_values(['Tahun', 'Bulan'], inplace=True)

# Encode Month to numeric
df_combined['Bulan_Num'] = df_combined['Bulan'].cat.codes + 1

# Remove empty value
df_combined.dropna(inplace=True)

# Preparing the data for model
X = df_combined[['Tahun', 'Bulan_Num']]
y = df_combined.drop(columns=['Bulan', 'Tahun', 'Bulan_Num'])

# Split data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pediction data for January - December 2025
pred_months = pd.DataFrame({'Tahun': [2025] * 12, 'Bulan_Num': list(range(1, 13))})

# Set the empy variable for the result of predictions
predictions = {}

# Traning and Evaluate the LightGBM Model

In [ ]:
# Fungsi untuk evaluasi model
def evaluate_model(true, pred, learning_rate=0.1):
    rmse = np.sqrt(mean_squared_error(true, pred))  # Root Mean Squared Error
    mae = mean_absolute_error(true, pred)          # Mean Absolute Error
    r2 = r2_score(true, pred)                      # R-squared
    re = true - pred                               # Residual Error (RE)
    y_new = pred + learning_rate * re              # Ybaru (LR update)
    return rmse, mae, r2, re, y_new

# Hyperparameter LightGBM
lgb_params = {
    'learning_rate': 0.1,
    'objective': 'regression',
    'metric': 'rmse',
    'verbose': -1
}

# Dictionary untuk menyimpan prediksi
lightgbm_preds = {}

# List untuk menyimpan hasil evaluasi semua rute
evaluation_results = []
rmse_list, mae_list, r2_list, re_list = [], [], [], []

# Loop untuk melatih model dan mengevaluasi tiap rute
for route in y.columns:
    # Latih model LightGBM untuk setiap rute
    lgb_train = lgb.Dataset(X, label=y[route])
    lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=100)

    # Simpan prediksi untuk data 2025
    lightgbm_preds[route] = lgb_model.predict(pred_months)

    # Evaluasi model menggunakan data uji (X_test, y_test)
    y_pred = lgb_model.predict(X_test)
    rmse, mae, r2, re, y_new = evaluate_model(y_test[route].values, y_pred)

    # Kumpulkan metrik untuk perhitungan rata-rata
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    re_list.extend(re)  # Kumpulkan semua residual error dari semua rute

    # Buat DataFrame hasil evaluasi
    result_df = pd.DataFrame({
        'Bulan': X_test['Bulan_Num'].values,
        'Y (Data Asli)': y_test[route].values,
        'Y (Prediksi)': y_pred,
        'Diperoleh dari': 'LightGBM',
        'Residual Error (RE)': re,
        'Ybaru (LR=0.1)': y_new,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    })

    # Urutkan hasil berdasarkan bulan
    result_df.sort_values('Bulan', inplace=True)

    # Simpan hasil evaluasi
    evaluation_results.append(result_df)

    # Tampilkan hasil prediksi dan evaluasi untuk setiap rute
    print(f"\n🔍 **Hasil Evaluasi untuk Rute: {route}**")
    display(result_df)

# Hitung rata-rata dari semua metrik
average_rmse = np.mean(rmse_list)
average_mae = np.mean(mae_list)
average_r2 = np.mean(r2_list)
average_re = np.mean(re_list)

# Tampilkan hasil rata-rata
print("\n📊 **Rata-rata Evaluasi Model LightGBM**")
print(f"📌 Rata-rata RMSE: {average_rmse:.4f}")
print(f"📌 Rata-rata MAE: {average_mae:.4f}")
print(f"📌 Rata-rata R²: {average_r2:.4f}")
print(f"📌 Rata-rata Residual Error (RE): {average_re:.4f}")

# Menampilkan semua prediksi LightGBM untuk 2025
lightgbm_pred_df = pd.DataFrame(lightgbm_preds, index=pred_months['Bulan_Num'])
lightgbm_pred_df.index.name = 'Bulan'
print("\n📊 **Prediksi 2025 menggunakan LightGBM**")
display(lightgbm_pred_df)


🔍 **Hasil Evaluasi untuk Rute: Belawan - Tanjung Balai Karimun**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,15200,18950.0,LightGBM,-3750.0,18575.0,2752.907554,2150.0,-0.059041
1,5,20500,18950.0,LightGBM,1550.0,19105.0,2752.907554,2150.0,-0.059041
3,7,23500,18950.0,LightGBM,4550.0,19405.0,2752.907554,2150.0,-0.059041
0,9,19800,18950.0,LightGBM,850.0,19035.0,2752.907554,2150.0,-0.059041
4,12,19000,18950.0,LightGBM,50.0,18955.0,2752.907554,2150.0,-0.059041



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Batam**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,18000,21708.333333,LightGBM,-3708.333333,21337.5,2628.701095,2075.0,-0.053364
1,5,23000,21708.333333,LightGBM,1291.666667,21837.5,2628.701095,2075.0,-0.053364
3,7,26000,21708.333333,LightGBM,4291.666667,22137.5,2628.701095,2075.0,-0.053364
0,9,22500,21708.333333,LightGBM,791.666667,21787.5,2628.701095,2075.0,-0.053364
4,12,22000,21708.333333,LightGBM,291.666667,21737.5,2628.701095,2075.0,-0.053364



🔍 **Hasil Evaluasi untuk Rute: Batam-Jakarta**


...

,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,17800,21450.0,LightGBM,-3650.0,21085.0,2599.711522,2070.0,-0.058264
1,5,22700,21450.0,LightGBM,1250.0,21575.0,2599.711522,2070.0,-0.058264
3,7,25700,21450.0,LightGBM,4250.0,21875.0,2599.711522,2070.0,-0.058264
0,9,22300,21450.0,LightGBM,850.0,21535.0,2599.711522,2070.0,-0.058264
4,12,21800,21450.0,LightGBM,350.0,21485.0,2599.711522,2070.0,-0.058264



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Belawan**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,14800,18420.833333,LightGBM,-3620.833333,18058.75,2733.209474,2147.5,-0.070017
1,5,20000,18420.833333,LightGBM,1579.166667,18578.75,2733.209474,2147.5,-0.070017
3,7,23000,18420.833333,LightGBM,4579.166667,18878.75,2733.209474,2147.5,-0.070017
0,9,19300,18420.833333,LightGBM,879.166667,18508.75,2733.209474,2147.5,-0.070017
4,12,18500,18420.833333,LightGBM,79.166667,18428.75,2733.209474,2147.5,-0.070017



📊 **Rata-rata Evaluasi Model LightGBM**
📌 Rata-rata RMSE: 2660.3142
📌 Rata-rata MAE: 2105.8333
📌 Rata-rata R²: -0.0613
📌 Rata-rata Residual Error (RE): 638.6111

📊 **Prediksi 2025 menggunakan LightGBM**


,Belawan - Tanjung Balai Karimun,Tanjung Balai Karimun - Batam,Batam-Jakarta,Jakarta - Batam,Batam - Tanjung Balai Karimun,Tanjung Balai Karimun - Belawan
Bulan,,,,,,
1,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
2,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
3,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
4,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
5,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
6,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
7,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
8,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333
9,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333


# Traning and Evaluate The CatBoost Model

In [ ]:
# Fungsi evaluasi model
def evaluate_model(true, pred, learning_rate=0.03):
    rmse = np.sqrt(mean_squared_error(true, pred))
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred)
    re = true - pred  # Residual Error (RE)
    y_new = pred + learning_rate * re  # Ybaru (LR update)
    return rmse, mae, r2, re, y_new

# Hyperparameter CatBoost
cat_params = {
    'iterations': 100,
    'learning_rate': 0.03,
    'loss_function': 'RMSE',
    'verbose': 0
}

# Dictionary untuk menyimpan prediksi CatBoost
catboost_preds = {}

# List untuk menyimpan hasil evaluasi semua rute
evaluation_results = []
rmse_list, mae_list, r2_list, re_list = [], [], [], []

# Loop untuk melatih model dan mengevaluasi tiap rute
for route in y.columns:
    # Latih model CatBoost untuk setiap rute
    cat_model = cb.CatBoostRegressor(**cat_params)
    cat_model.fit(X, y[route])

    # Simpan prediksi untuk data 2025
    catboost_preds[route] = cat_model.predict(pred_months)

    # Evaluasi model menggunakan data uji (X_test, y_test)
    y_pred = cat_model.predict(X_test)
    rmse, mae, r2, re, y_new = evaluate_model(y_test[route].values, y_pred)

    # Kumpulkan metrik untuk perhitungan rata-rata
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    re_list.extend(re)  # Kumpulkan semua residual error dari semua rute

    # Buat DataFrame hasil evaluasi
    result_df = pd.DataFrame({
        'Bulan': X_test['Bulan_Num'].values,
        'Y (Data Asli)': y_test[route].values,
        'Y (Prediksi)': y_pred,
        'Diperoleh dari': 'CatBoost',
        'Residual Error (RE)': re,
        'Ybaru (LR=0.03)': y_new,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    })

    # Hanya tampilkan data dengan nilai asli yang tidak kosong
    result_df.dropna(subset=['Y (Data Asli)'], inplace=True)

    # Urutkan hasil berdasarkan bulan
    result_df.sort_values('Bulan', inplace=True)

    # Simpan hasil evaluasi ke dalam list
    evaluation_results.append(result_df)

    # Tampilkan hasil prediksi dan evaluasi untuk setiap rute
    print(f"\n🔍 **Hasil Evaluasi untuk Rute: {route}**")
    display(result_df)

# Hitung rata-rata dari semua metrik
average_rmse = np.mean(rmse_list)
average_mae = np.mean(mae_list)
average_r2 = np.mean(r2_list)
average_re = np.mean(re_list)

# Tampilkan hasil rata-rata
print("\n📊 **Rata-rata Evaluasi Model CatBoost**")
print(f"📌 Rata-rata RMSE: {average_rmse:.4f}")
print(f"📌 Rata-rata MAE: {average_mae:.4f}")
print(f"📌 Rata-rata R²: {average_r2:.4f}")
print(f"📌 Rata-rata Residual Error (RE): {average_re:.4f}")

# Menampilkan semua prediksi CatBoost untuk 2025
print("\n📊 **Prediksi 2025 menggunakan CatBoost**")
catboost_pred_df = pd.DataFrame(catboost_preds, index=pred_months['Bulan_Num'])
catboost_pred_df.index.name = 'Bulan'
display(catboost_pred_df)


🔍 **Hasil Evaluasi untuk Rute: Belawan - Tanjung Balai Karimun**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.03),RMSE,MAE,R²
2,1,15200,16105.238399,CatBoost,-905.238399,16078.081247,941.952002,670.274509,0.87601
1,5,20500,20532.632501,CatBoost,-32.632501,20531.653526,941.952002,670.274509,0.87601
3,7,23500,21669.909824,CatBoost,1830.090176,21724.812530,941.952002,670.274509,0.87601
0,9,19800,19727.875582,CatBoost,72.124418,19730.039314,941.952002,670.274509,0.87601
4,12,19000,18488.712951,CatBoost,511.287049,18504.051562,941.952002,670.274509,0.87601



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Batam**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.03),RMSE,MAE,R²
2,1,18000,18946.048877,CatBoost,-946.048877,18917.667410,905.777447,669.869389,0.874934
1,5,23000,22959.548645,CatBoost,40.451355,22960.762186,905.777447,669.869389,0.874934
3,7,26000,24308.918678,CatBoost,1691.081322,24359.651117,905.777447,669.869389,0.874934
0,9,22500,22409.197615,CatBoost,90.802385,22411.921686,905.777447,669.869389,0.874934
4,12,22000,21419.036995,CatBoost,580.963005,21436.465885,905.777447,669.869389,0.874934



🔍 **Hasil Evaluasi untuk Rute: Batam-Jakarta**


...

,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.03),RMSE,MAE,R²
2,1,17800,18676.735744,CatBoost,-876.735744,18650.433671,889.51555,659.095561,0.876106
1,5,22700,22634.901760,CatBoost,65.098240,22636.854708,889.51555,659.095561,0.876106
3,7,25700,24026.995159,CatBoost,1673.004841,24077.185304,889.51555,659.095561,0.876106
0,9,22300,22236.001261,CatBoost,63.998739,22237.921223,889.51555,659.095561,0.876106
4,12,21800,21183.359759,CatBoost,616.640241,21201.858967,889.51555,659.095561,0.876106



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Belawan**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.03),RMSE,MAE,R²
2,1,14800,15628.458818,CatBoost,-828.458818,15603.605054,927.02125,679.250585,0.87691
1,5,20000,19866.048252,CatBoost,133.951748,19870.066805,927.02125,679.250585,0.87691
3,7,23000,21208.799319,CatBoost,1791.200681,21262.535339,927.02125,679.250585,0.87691
0,9,19300,19323.276493,CatBoost,-23.276493,19322.578199,927.02125,679.250585,0.87691
4,12,18500,17880.634816,CatBoost,619.365184,17899.215772,927.02125,679.250585,0.87691



📊 **Rata-rata Evaluasi Model CatBoost**
📌 Rata-rata RMSE: 912.1254
📌 Rata-rata MAE: 667.4633
📌 Rata-rata R²: 0.8752
📌 Rata-rata Residual Error (RE): 305.7491

📊 **Prediksi 2025 menggunakan CatBoost**


,Belawan - Tanjung Balai Karimun,Tanjung Balai Karimun - Batam,Batam-Jakarta,Jakarta - Batam,Batam - Tanjung Balai Karimun,Tanjung Balai Karimun - Belawan
Bulan,,,,,,
1,16442.556107,19296.813105,14994.374729,14955.461120,18955.461120,15935.119894
2,16331.729171,19244.282440,14906.874419,14888.690509,18888.690509,15828.716355
3,17080.606660,19895.642874,15215.782880,15639.707275,19639.707275,16464.627806
4,18989.513043,21541.499321,17306.546702,17247.970838,21247.970838,18412.728896
5,20532.632501,22959.548645,18969.642416,18634.901760,22634.901760,19866.048252
6,21404.224091,23953.460925,19867.186375,19649.268770,23649.268770,20778.859434
7,21669.909824,24308.918678,20287.960511,20026.995159,24026.995159,21208.799319
8,21458.683825,24093.210698,20086.151927,19800.054932,23800.054932,21063.204148
9,20110.049232,22805.578390,18880.946487,18546.768703,22546.768703,19693.195314


# Traning and Evaluate The XGBoost Model

In [ ]:
# Fungsi evaluasi model
def evaluate_model(true, pred, learning_rate=0.1):
    rmse = np.sqrt(mean_squared_error(true, pred))
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred)
    re = true - pred  # Residual Error (RE)
    y_new = pred + learning_rate * re  # Ybaru (LR update)
    return rmse, mae, r2, re, y_new

# Hyperparameter XGBoost
xgb_params = {'learning_rate': 0.1, 'objective': 'reg:squarederror'}

# Dictionary untuk menyimpan prediksi XGBoost
xgboost_preds = {}

# List untuk menyimpan hasil evaluasi semua rute
evaluation_results = []
rmse_list, mae_list, r2_list, re_list = [], [], [], []

# Loop untuk melatih model dan mengevaluasi tiap rute
for route in y.columns:
    # Latih model XGBoost untuk setiap rute
    xgb_train = xgb.DMatrix(X, label=y[route])
    xgb_model = xgb.train(xgb_params, xgb_train, num_boost_round=100)

    # Simpan prediksi untuk data 2025
    xgboost_preds[route] = xgb_model.predict(xgb.DMatrix(pred_months))

    # Evaluasi model menggunakan data uji (X_test, y_test)
    y_pred = xgb_model.predict(xgb.DMatrix(X_test))
    rmse, mae, r2, re, y_new = evaluate_model(y_test[route].values, y_pred)

    # Kumpulkan metrik untuk perhitungan rata-rata
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    re_list.extend(re)  # Gabungkan residual error dari semua rute

    # Buat DataFrame hasil evaluasi
    result_df = pd.DataFrame({
        'Bulan': X_test['Bulan_Num'].values,
        'Y (Data Asli)': y_test[route].values,
        'Y (Prediksi)': y_pred,
        'Diperoleh dari': 'XGBoost',
        'Residual Error (RE)': re,
        'Ybaru (LR=0.1)': y_new,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    })

    # Hanya tampilkan baris yang memiliki Y (Data Asli)
    result_df.dropna(subset=['Y (Data Asli)'], inplace=True)

    # Urutkan hasil berdasarkan bulan
    result_df.sort_values('Bulan', inplace=True)

    # Simpan hasil evaluasi
    evaluation_results.append(result_df)

    # Tampilkan hasil prediksi dan evaluasi untuk setiap rute
    print(f"\n🔍 **Hasil Evaluasi untuk Rute: {route}**")
    display(result_df)

# Hitung rata-rata dari semua metrik
average_rmse = np.mean(rmse_list)
average_mae = np.mean(mae_list)
average_r2 = np.mean(r2_list)
average_re = np.mean(re_list)

# Tampilkan hasil rata-rata
print("\n📊 **Rata-rata Evaluasi Model XGBoost**")
print(f"📌 Rata-rata RMSE: {average_rmse:.4f}")
print(f"📌 Rata-rata MAE: {average_mae:.4f}")
print(f"📌 Rata-rata R²: {average_r2:.4f}")
print(f"📌 Rata-rata Residual Error (RE): {average_re:.4f}")

# Menampilkan semua prediksi XGBoost untuk 2025
print("\n📊 **Prediksi 2025 menggunakan XGBoost**")
xgboost_pred_df = pd.DataFrame(xgboost_preds, index=pred_months['Bulan_Num'])
xgboost_pred_df.index.name = 'Bulan'
display(xgboost_pred_df)



🔍 **Hasil Evaluasi untuk Rute: Belawan - Tanjung Balai Karimun**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,15200,15205.730469,XGBoost,-5.730469,15205.157422,11.222493,6.371875,0.999982
1,5,20500,20498.841797,XGBoost,1.158203,20498.957617,11.222493,6.371875,0.999982
3,7,23500,23475.599609,XGBoost,24.400391,23478.039648,11.222493,6.371875,0.999982
0,9,19800,19799.728516,XGBoost,0.271484,19799.755664,11.222493,6.371875,0.999982
4,12,19000,18999.701172,XGBoost,0.298828,18999.731055,11.222493,6.371875,0.999982



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Batam**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,18000,18005.876953,XGBoost,-5.876953,18005.289258,11.270401,7.098828,0.999981
1,5,23000,22996.740234,XGBoost,3.259766,22997.066211,11.270401,7.098828,0.999981
3,7,26000,25975.757812,XGBoost,24.242188,25978.182031,11.270401,7.098828,0.999981
0,9,22500,22499.052734,XGBoost,0.947266,22499.147461,11.270401,7.098828,0.999981
4,12,22000,21998.832031,XGBoost,1.167969,21998.948828,11.270401,7.098828,0.999981



🔍 **Hasil Evaluasi untuk Rute: Batam-Jakarta**


...

,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,17800,17805.349609,XGBoost,-5.349609,17804.814648,10.506296,6.182031,0.999983
1,5,22700,22698.705078,XGBoost,1.294922,22698.834570,10.506296,6.182031,0.999983
3,7,25700,25677.193359,XGBoost,22.806641,25679.474023,10.506296,6.182031,0.999983
0,9,22300,22299.722656,XGBoost,0.277344,22299.750391,10.506296,6.182031,0.999983
4,12,21800,21798.818359,XGBoost,1.181641,21798.936523,10.506296,6.182031,0.999983



🔍 **Hasil Evaluasi untuk Rute: Tanjung Balai Karimun - Belawan**


,Bulan,Y (Data Asli),Y (Prediksi),Diperoleh dari,Residual Error (RE),Ybaru (LR=0.1),RMSE,MAE,R²
2,1,14800,14805.720703,XGBoost,-5.720703,14805.148633,11.272439,6.688281,0.999982
1,5,20000,19997.482422,XGBoost,2.517578,19997.734180,11.272439,6.688281,0.999982
3,7,23000,22975.587891,XGBoost,24.412109,22978.029102,11.272439,6.688281,0.999982
0,9,19300,19299.677734,XGBoost,0.322266,19299.709961,11.272439,6.688281,0.999982
4,12,18500,18499.531250,XGBoost,0.468750,18499.578125,11.272439,6.688281,0.999982



📊 **Rata-rata Evaluasi Model XGBoost**
📌 Rata-rata RMSE: 10.9810
📌 Rata-rata MAE: 6.5641
📌 Rata-rata R²: 1.0000
📌 Rata-rata Residual Error (RE): 4.1735

📊 **Prediksi 2025 menggunakan XGBoost**


,Belawan - Tanjung Balai Karimun,Tanjung Balai Karimun - Batam,Batam-Jakarta,Jakarta - Batam,Batam - Tanjung Balai Karimun,Tanjung Balai Karimun - Belawan
Bulan,,,,,,
1,15801.818359,18501.648438,14499.750000,14300.864258,18300.865234,15301.860352
2,14807.967773,18008.433594,13805.177734,13507.781250,17507.781250,14307.875000
3,16501.207031,19501.500000,13814.807617,15201.262695,19201.263672,16001.334961
4,18999.701172,21500.685547,17500.763672,17200.687500,21200.687500,18300.589844
5,20498.841797,22996.740234,18998.644531,18698.705078,22698.705078,19997.482422
6,21995.554688,24495.244141,20494.880859,20194.884766,24194.884766,21295.603516
7,23475.599609,25975.757812,21976.505859,21677.193359,25677.193359,22975.587891
8,22994.181641,25495.818359,21494.806641,21195.718750,25195.718750,22494.820312
9,20198.531250,22998.939453,18997.257812,18698.583984,22698.583984,19698.748047


### Show the result

In [ ]:
# Gabungkan semua prediksi ke dalam satu DataFrame
combined_preds = {}

# Tambahkan prediksi LightGBM
temp_lgbm = {f"LightGBM_{route}": lightgbm_preds[route] for route in lightgbm_preds}
combined_preds.update(temp_lgbm)

# Tambahkan prediksi CatBoost
temp_catboost = {f"CatBoost_{route}": catboost_preds[route] for route in catboost_preds}
combined_preds.update(temp_catboost)

# Tambahkan prediksi XGBoost
temp_xgboost = {f"XGBoost_{route}": xgboost_preds[route] for route in xgboost_preds}
combined_preds.update(temp_xgboost)

# Buat DataFrame dari semua prediksi
combined_preds_df = pd.DataFrame(combined_preds, index=pred_months['Bulan_Num'])
combined_preds_df.index.name = 'Bulan'

# Tampilkan hasil akhir
print("\n📊 **Gabungan Prediksi Penjualan Tiket Kapal pada 2025 dari LightGBM, CatBoost, dan XGBoost**")
display(combined_preds_df)


📊 **Gabungan Prediksi Penjualan Tiket Kapal pada 2025 dari LightGBM, CatBoost, dan XGBoost**


,LightGBM_Belawan - Tanjung Balai Karimun,LightGBM_Tanjung Balai Karimun - Batam,LightGBM_Batam-Jakarta,LightGBM_Jakarta - Batam,LightGBM_Batam - Tanjung Balai Karimun,LightGBM_Tanjung Balai Karimun - Belawan,CatBoost_Belawan - Tanjung Balai Karimun,CatBoost_Tanjung Balai Karimun - Batam,CatBoost_Batam-Jakarta,CatBoost_Jakarta - Batam,CatBoost_Batam - Tanjung Balai Karimun,CatBoost_Tanjung Balai Karimun - Belawan,XGBoost_Belawan - Tanjung Balai Karimun,XGBoost_Tanjung Balai Karimun - Batam,XGBoost_Batam-Jakarta,XGBoost_Jakarta - Batam,XGBoost_Batam - Tanjung Balai Karimun,XGBoost_Tanjung Balai Karimun - Belawan
Bulan,,,,,,,,,,,,,,,,,,
1,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,16442.556107,19296.813105,14994.374729,14955.461120,18955.461120,15935.119894,15801.818359,18501.648438,14499.750000,14300.864258,18300.865234,15301.860352
2,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,16331.729171,19244.282440,14906.874419,14888.690509,18888.690509,15828.716355,14807.967773,18008.433594,13805.177734,13507.781250,17507.781250,14307.875000
3,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,17080.606660,19895.642874,15215.782880,15639.707275,19639.707275,16464.627806,16501.207031,19501.500000,13814.807617,15201.262695,19201.263672,16001.334961
4,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,18989.513043,21541.499321,17306.546702,17247.970838,21247.970838,18412.728896,18999.701172,21500.685547,17500.763672,17200.687500,21200.687500,18300.589844
5,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,20532.632501,22959.548645,18969.642416,18634.901760,22634.901760,19866.048252,20498.841797,22996.740234,18998.644531,18698.705078,22698.705078,19997.482422
6,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,21404.224091,23953.460925,19867.186375,19649.268770,23649.268770,20778.859434,21995.554688,24495.244141,20494.880859,20194.884766,24194.884766,21295.603516
7,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,21669.909824,24308.918678,20287.960511,20026.995159,24026.995159,21208.799319,23475.599609,25975.757812,21976.505859,21677.193359,25677.193359,22975.587891
8,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,21458.683825,24093.210698,20086.151927,19800.054932,23800.054932,21063.204148,22994.181641,25495.818359,21494.806641,21195.718750,25195.718750,22494.820312
9,18950.0,21708.333333,17629.166667,17450.0,21450.0,18420.833333,20110.049232,22805.578390,18880.946487,18546.768703,22546.768703,19693.195314,20198.531250,22998.939453,18997.257812,18698.583984,22698.583984,19698.748047
